In [32]:
import numpy as np
import json
import pandas as pd
import os
import re
import ast
import xml.etree.ElementTree as ET
import ast

from __future__ import print_function
from gensim import corpora

from nltk.sem.logic import *
from nltk.sem.logic import LogicParser

In [33]:
#ここを外部読み込みにさせたい.
variable_arr = []
pre_var = []
formulas = []
check_dup = set([])

from nltk.sem.logic import LogicParser
from nltk.sem.logic import *

logic_parser = LogicParser(type_check=False)
def lexpr(formula_str):
    return logic_parser.parse(formula_str)


def coq_string_expr(expression):
    org_exp = expression
    
    if isinstance(expression, str):
        expression = lexpr(expression)
    expr_coq_str = ''
    if isinstance(expression, ApplicationExpression):
        coq_string_application_expr(expression)
    elif isinstance(expression, AbstractVariableExpression):
        expr_coq_str = coq_string_abstract_variable_expr(expression)
    elif isinstance(expression, LambdaExpression):
        raise
        #return 'lambda_expression'
    elif isinstance(expression, QuantifiedExpression):
        expr_coq_str = coq_string_quantified_expr(expression)
    elif isinstance(expression, AndExpression):
        expr_coq_str = coq_string_and_expr(expression)
    elif isinstance(expression, OrExpression):
        expr_coq_str = coq_string_or_expr(expression)
    elif isinstance(expression, NegatedExpression):
        expr_coq_str = coq_string_not_expr(expression)
    elif isinstance(expression, BinaryExpression):
        expr_coq_str = coq_string_binary_expr(expression)
    elif isinstance(expression, Variable):
        expr_coq_str = '%s' % expression
    else:
        expr_coq_str = str(expression)

    return org_exp

coqstr = coq_string_expr

def coq_string_application_expr(expression):
    # uncurry the arguments and find the base function
    if expression.is_atom():
        #is_atom : 原子論理式かどうか
        function, args = expression.uncurry()
        arg_str = ' '.join("%s" % coqstr(arg) for arg in args)
    else:
        #Leave arguments curried
        function = expression.function
        arg_str = "%s" % coqstr(expression.argument)
    function_str = "%s" % coqstr(function)
    parenthesize_function = False
    if isinstance(function, LambdaExpression):
        if isinstance(function.term, ApplicationExpression):
            if not isinstance(function.term.function,
                              AbstractVariableExpression):
                parenthesize_function = True
        elif not isinstance(function.term, BooleanExpression):
            parenthesize_function = True
    elif isinstance(function, ApplicationExpression):
        parenthesize_function = True
    if parenthesize_function:
        function_str = Tokens.OPEN + function_str + Tokens.CLOSE
    return expression 

def coq_string_abstract_variable_expr(expression):
    expr_str = str(expression.variable)
    if not isinstance(expression, FunctionVariableExpression):
        if expr_str == '':
            expr_str = "%s" % expr_str
        else:
            expr_str = "%s" % expr_str
    else:
        expr_str = "%s" % expr_str
        
    if not re.sub(r'_', "",expr_str) in check_dup:
        if not expr_str.startswith('_'):
            pre_var.append(expr_str)
            check_dup.add(expr_str)
    return expression


def coq_string_quantified_expr(expression):
    variables = [expression.variable]
    term = expression.term
    while term.__class__ == expression.__class__:
        variables.append(term.variable)
        term = term.term
    for v in variables:
        coqstr(v)
    coqstr(term) 
    return expression

def coq_string_and_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return expression 

def coq_string_or_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return expression

def coq_string_not_expr(expression):
    term_str = coqstr(expression.term)
    return expression

def coq_string_binary_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return expression


In [34]:
def substituteString(text,lst):
    ###辞書作り###
    lst = list(set(lst))
    e_sub = {}
    z_sub = {}
    ice = 0 #index counter
    icz = 0 

    for i in lst:
        matchObj = re.search(r'e0+', i)
        if(matchObj):
            tmp = "e0"+str(ice)
            e_sub[tmp] = i
            ice+=1
            continue
        matchObj = re.search(r'z[0-9]*', i)
        if matchObj:
            tmp = "z0"+str(icz)
            z_sub[tmp] = i
            icz+=1
    
    e_sub = {v:k for k, v in e_sub.items()}
    z_sub = {v:k for k, v in z_sub.items()}
    
    for k, v in e_sub.items():  
        text = text.replace(k,v)
    for k, v in z_sub.items():  
        text = text.replace(k,v)
    return text

In [37]:
#en_parsed のsemから取り出し ただし，論理記号の順番がスタック式をなおさなければならなそう
tree = ET.parse('sem/snli_dev.txt.candc.sem.xml')
root = tree.getroot()

root = root[0]
root = root[0] #１個目のsentence

mydict = {}
#c = 0 #辞書のindexを回す
print(len(root))

#子階層のタグと中身
for c,child in enumerate(root):
    
    formula = child[2]  #child[2]がsemantics
    check = (child[2].attrib)
    check = check['status']
   
    if(check == 'success'):
        plain = ""
        toridashi = child[0]
        
        for i in toridashi :
            p = i.attrib
            p = p['surf']
            if(p=='.' or p== ','):
                plain = plain + p
            else :
                plain = plain + " " + p
        plain = plain+'\n'
        
        #print(plain)
        
        formula = child[2][0]
        formula = (formula.attrib)
        formula = formula['sem'] #\nが付与されていると信じている
        
        #論理式を今回の標準にする
        check_dup = set([])
        pre_var = []
        
        try:
            org_f = formula
            formula = coq_string_expr(formula)
        except:
            print(c,'except: ',org_f,'\n')
            import traceback
            traceback.print_exc()
            continue
            #pre_var = []
            
        if(formula == 'lambda_expression'):
            print(c,'lambda: ',org_f,'\n')
            continue
        formula = substituteString(formula,pre_var)
        ice = 0
        icz = 0

        for i in range(len(pre_var)):
            matchObj = re.search(r'e0+', pre_var[i])
            if(matchObj):
                tmp = "e0"+str(ice)
                pre_var[i] = tmp

            matchObj = re.search(r'z[0-9]*', pre_var[i])
            if matchObj:
                tmp = "z0"+str(icz)
                pre_var[i] = tmp
                icz+=1

        #variable_arr.append(pre_var)
        #pre_var = []
        pair = {'text':plain,'formula':formula}
        mydict.update({str(c):pair})
            
    else:
      print('faild parse')
    #c+=1
    #if(c==10):
    #    break
print(len(mydict))
    
#f = open('snli_input_data_1214.json', 'w') # 書き込みモードで開く
#json.dump(mydict, f,ensure_ascii=False)
#f.close()

19949
96 except:  \X0 K.X0(\F2 F3.exists x.(_car(x) & F2(x) & F3(x)),\e.(_break(e) & _down(e) & exists x.(_side(x) & exists z381.(_road(z381) & TrueP & (x = z381)) & TrueP & _on(e,x) & K(e)))) 

106 except:  \X0 Q K.Q(\x.TrueP,\x.(X0(\G1 G2.G2(x),\F2 F3.exists z411.(exists e.(_blond(e) & (Subj(e) = z411) & _laugh(z411) & _man(z411)) & F2(z411) & F3(z411)),\e.TrueP) & exists z408.(_laundry(z408) & TrueP & exists e.(_do(e) & (Subj(e) = x) & (Acc(e) = z408) & K(e))))) 

140 except:  \X0 Q K.(exists x.(_race(x) & _suv(x) & _taxi(x) & X0(x) & \K.(x(\z581.TrueP,\z582.exists z580.(_client(z580) & TrueP & exists e.(_pick(e) & (Subj(e) = z582) & (Acc(e) = z580) & _up(e) & TrueP))) & exists z583.(_man(z583) & TrueP & exists e.(_walk(e) & (Subj(e) = z583) & _peacefully(e) & exists z583.(_direction(z583) & TrueP & _in(e,z583) & TrueP)))))(Q))(K) 



Traceback (most recent call last):
  File "<ipython-input-37-323f68e66ca2>", line 44, in <module>
    formula = coq_string_expr(formula)
  File "<ipython-input-33-ad8d6eba4d27>", line 26, in coq_string_expr
    raise
RuntimeError: No active exception to reraise
Traceback (most recent call last):
  File "<ipython-input-37-323f68e66ca2>", line 44, in <module>
    formula = coq_string_expr(formula)
  File "<ipython-input-33-ad8d6eba4d27>", line 26, in coq_string_expr
    raise
RuntimeError: No active exception to reraise
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 142, in parse
    result = self.process_next_expression(None)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 273, in process_next_expression
    accum = self.handle(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 290, in handle
    return self.handle_lambda(tok, context)
  File "/usr/local/lib/python3.6/s

597 except:  \X0 Q K.(X0(Q,\F1 F2.exists x.(_goalie(x) & exists z1857.(exists e.(_white(e) & (Subj(e) = z1857) & _run(z1857)) & exists z1856.(_ball(z1856) & _approach(z1856) & TrueP & (z1857 = z1856)) & TrueP & exists e.(_in(e,z1857) & (Subj(e) = x) & F1(x))) & F2(x)),\e.TrueP) & exists x.(_opponent(x) & exists z1858.(_red(z1858) & TrueP & exists e.(_in(e,z1858) & (Subj(e) = x) & exists z1859.((z1859 = _it) & TrueP & exists e.(_kick(e) & (Subj(e) = x) & (Acc(e) = z1859) & TrueP)) & TrueP)) & exists e.(_wait(e) & (Subj(e) = x) & TrueP))) 

599 except:  \X0 Q K.(X0(Q,\F1 F2.exists x.(_goalie(x) & exists z1868.(exists e.(_white(e) & (Subj(e) = z1868) & _run(z1868)) & exists z1867.(_ball(z1867) & _approach(z1867) & TrueP & (z1868 = z1867)) & TrueP & exists e.(_in(e,z1868) & (Subj(e) = x) & F1(x))) & F2(x)),\e.TrueP) & exists x.(_opponent(x) & exists z1869.(_red(z1869) & TrueP & exists e.(_in(e,z1869) & (Subj(e) = x) & exists z1870.((z1870 = _it) & TrueP & exists e.(_kick(e) & (Subj(e) = x)

  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 438, in attempt_BooleanExpression
    self.process_next_expression(tok))
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 273, in process_next_expression
    accum = self.handle(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 290, in handle
    return self.handle_lambda(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 363, in handle_lambda
    accum = self.process_next_expression(tok)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 273, in process_next_expression
    accum = self.handle(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 296, in handle
    return self.handle_open(tok, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 406, in handle_open
    accum = self.process_next_expression(None)
  File "/usr/local/lib/python3.6

802 except:  \X0 Q K.Q(\x.TrueP,\x.(_all(x) & X0(\F1 F2.F2(x),\F2 F3.exists z2404.(_crew(z2404) & F2(z2404) & F3(z2404)),\e.(_alone(e) & K(e))) & exists e.(_ate(e) & (Subj(e) = x) & _lunch(x)))) 

879 except:  \x.(_cow(x) & _herding(x) & _dog(x)) 

881 except:  \x.(_cow(x) & _herding(x) & _dog(x)) 

883 except:  \x.(_cow(x) & _herding(x) & _dog(x)) 



...

Traceback (most recent call last):
  File "<ipython-input-37-323f68e66ca2>", line 44, in <module>
    formula = coq_string_expr(formula)
  File "<ipython-input-33-ad8d6eba4d27>", line 26, in coq_string_expr
    raise
RuntimeError: No active exception to reraise
Traceback (most recent call last):
  File "<ipython-input-37-323f68e66ca2>", line 44, in <module>
    formula = coq_string_expr(formula)
  File "<ipython-input-33-ad8d6eba4d27>", line 26, in coq_string_expr
    raise
RuntimeError: No active exception to reraise


19585 except:  (_out(\x.TrueP) & exists x.(exists e057907.(_gray(e057907) & (Subj(e057907) = x) & _roof(x)) & TrueP & _on(\x.TrueP,x)))(\e.TrueP) 



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 142, in parse
    result = self.process_next_expression(None)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 278, in process_next_expression
    return self.attempt_adjuncts(accum, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 303, in attempt_adjuncts
    expression = self.attempt_ApplicationExpression(expression, context)
  File "/usr/local/lib/python3.6/site-packages/nltk/sem/logic.py", line 473, in attempt_ApplicationExpression
    "' is not a Lambda Expression, an "
nltk.sem.logic.LogicalExpressionException: The function '(_out(\x.TrueP) & exists x.(exists e057907.(_gray(e057907) & (Subj(e057907) = x) & _roof(x)) & TrueP & _on(\x.TrueP,x)))' is not a Lambda Expression, an Application Expression, or a functional predicate, so it may not take arguments.

During handling of the above exception, another exception occurred

19816 except:  \x.(exists e.(_small(e) & (Subj(e) = x) & exists e.(_open(e) & (Subj(e) = x) & _present(x)) & _child(x)) & _two(x)) 

19943 except:  \V2 Q K.(V2(Q,\F2 F3.exists x.(_woman(x) & F2(x) & F3(x)),\e.(exists x.(exists e059087.(_white(e059087) & (Subj(e059087) = x) & _t_dash_shirt(x)) & TrueP & _with(e,x) & K(e)) & exists x.(_skirt(x) & _zebra(x) & exists e059088.(_wade(e059088) & (Subj(e059088) = x) & exists z59086.(exists e059084.(_public(e059084) & (Subj(e059084) = z59086) & _fountain(z59086) & _water(z59086)) & TrueP & _in(e059088,z59086) & TrueP)) & TrueP & _with(e,x) & K(e)))) & V2(Q,\F1 F2.exists x.(exists e.(_young(e) & (Subj(e) = x) & _girl(x)) & exists z59083.(exists e.(_pink(e) & (Subj(e) = z59083) & exists e.(_blue(e) & (Subj(e) = z59083) & _ribbon(z59083))) & exists z59082.(_braid(z59082) & TrueP & exists e.(_in(e,z59082) & (Subj(e) = z59083) & TrueP)) & exists e.(_with(e,z59083) & (Subj(e) = x) & F1(x))) & F2(x)),\e.exists x.(_top(x) & TrueP & _without(e,x) & K(e)

Traceback (most recent call last):
  File "<ipython-input-37-323f68e66ca2>", line 44, in <module>
    formula = coq_string_expr(formula)
  File "<ipython-input-33-ad8d6eba4d27>", line 26, in coq_string_expr
    raise
RuntimeError: No active exception to reraise
Traceback (most recent call last):
  File "<ipython-input-37-323f68e66ca2>", line 44, in <module>
    formula = coq_string_expr(formula)
  File "<ipython-input-33-ad8d6eba4d27>", line 26, in coq_string_expr
    raise
RuntimeError: No active exception to reraise
Traceback (most recent call last):
  File "<ipython-input-37-323f68e66ca2>", line 44, in <module>
    formula = coq_string_expr(formula)
  File "<ipython-input-33-ad8d6eba4d27>", line 26, in coq_string_expr
    raise
RuntimeError: No active exception to reraise
Traceback (most recent call last):
  File "<ipython-input-37-323f68e66ca2>", line 44, in <module>
    formula = coq_string_expr(formula)
  File "<ipython-input-33-ad8d6eba4d27>", line 26, in coq_string_expr
    rai

In [38]:
print(len(mydict))

19587
